<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/Kaggle_Toxic_Comments_Feb23/Feb25-Model-7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
print("Name of GPU : {}".format(torch.cuda.get_device_name(0)))
print("# of GPU : {}".format(torch.cuda.device_count()))


Name of GPU : Tesla K80
# of GPU : 1


In [0]:
import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
import torch
from torchtext import data
import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
import zipfile
import gzip

tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize
import torch
import torch.nn as nn
import random
import keras
import numpy as np
from keras.layers import Conv1D, Conv2D, Input, SpatialDropout1D, Embedding, Dense, Activation, Bidirectional, LSTM,CuDNNLSTM, CuDNNGRU, GlobalMaxPooling1D,GlobalAveragePooling1D, concatenate, Dropout
from keras import optimizers
from keras.optimizers import Adam, RMSprop
from keras.models import Model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.engine import Layer
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection,linear_model,metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from torch.optim.optimizer import Optimizer
from unidecode import unidecode

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/kaggle_toxic_comments"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
best_model.hdf5			   test.csv
ext_fasttextwiki_embeddings.npy    test_features.npy
ext_glove_embeddings.npy	   test_labels.csv
ext_lexvec_embeddings.npy	   toxic_lookup_en_wiki_fasttext_300d.npy
features.npy			   train.csv
inputed_lexvec_embeddings.npy	   word_index.npy
local_fasttextwiki_embeddings.npy  x_test.npy
Model-runs			   x_train.npy
sample_submission.csv		   y_train.npy
submission_baseline_lr.csv


In [0]:
#configs
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector) ##sum([word[1] for word in vocab[:120000]])=92%##
maxlen = 250 # max number of words in a question to use ## train.num_words.quantile(q=0.96, interpolation='nearest')=258 ##
batch_size = 512 # how many samples to process at once
n_epochs = 5 # how many times to iterate over all samples
n_splits = 5 # Number of K-fold Splits

SEED = 2019
def seed_everything(seed=2019):
    random.seed(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()


In [0]:
os.chdir("/content/drive/My Drive/kaggle_toxic_comments/")
x_train = np.load("x_train.npy")
x_test = np.load("x_test.npy")
y_train = np.load("y_train.npy")
features = np.load("features.npy")
test_features = np.load("test_features.npy")
word_index = np.load("word_index.npy").item()
ext_lexvec_embeddings = np.load("ext_lexvec_embeddings.npy")

In [0]:
# Train simple LSTM using different embeddings
# Top compeitions- QIQC and Toxic comments
# ensembles (stacking and blending)
# Pretrained models- ULMFit and BERT
# Gensim and fasttext text classification
# Other SOTAs

In [0]:
#https://www.kaggle.com/c/quora-insincere-questions-classification/kernels?sortBy=scoreDescending&group=everyone&pageSize=20&competitionId=10737


In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size = 0.1)

In [0]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

            
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=4590., mode='triangular',gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}
        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):     
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v) 
        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
inp = Input(shape=(maxlen,))
x   = Embedding(max_features, embed_size, weights=[ext_lexvec_embeddings], trainable=False)(inp)
x   = SpatialDropout1D(rate=0.2)(x)
x   = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
x   = Conv1D(64, kernel_size=1, padding='valid', kernel_initializer='glorot_normal')(x)
x1  = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
x1   = Conv1D(64, kernel_size=1, padding='valid', kernel_initializer='glorot_normal')(x1)
x_max   = GlobalMaxPooling1D()(x)
x_avg   = GlobalAveragePooling1D()(x)
x1_max  = GlobalMaxPooling1D()(x1)
x1_avg  = GlobalAveragePooling1D()(x1)
c   = concatenate([x_max,x_avg,x1_max,x1_avg])
y   = Dense(64, activation='relu')(c)
y   = Dropout(0.2)(y)
out = Dense(6, activation='sigmoid')(y)
model=Model(inputs=inp,outputs=out)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-3), metrics=["accuracy"])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 250, 300)     36000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 250, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidi

In [0]:
# for one cycle policy: step_size=(len(X_train)/batch_size)*n_ecpochs/2
clr = CyclicLR(base_lr=1e-6, max_lr=1e-3,step_size=2805., mode='exp_range',gamma=0.99994)
model.fit(X_train, Y_train, batch_size = 128, epochs = 5, validation_data = (X_valid, Y_valid), verbose = 1, callbacks = [clr, ra_val, check_point, early_stop])


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 286s 2ms/step - loss: 0.0659 - acc: 0.9774 - val_loss: 0.0473 - val_acc: 0.9819

 ROC-AUC - epoch: 1 - score: 0.974741

Epoch 00001: val_loss improved from inf to 0.04727, saving model to best_model.hdf5
Epoch 2/5
143613/143613 [==============================] - 284s 2ms/step - loss: 0.0496 - acc: 0.9816 - val_loss: 0.0453 - val_acc: 0.9826

 ROC-AUC - epoch: 2 - score: 0.979609

Epoch 00002: val_loss improved from 0.04727 to 0.04528, saving model to best_model.hdf5
Epoch 3/5
143613/143613 [==============================] - 284s 2ms/step - loss: 0.0460 - acc: 0.9824 - val_loss: 0.0424 - val_acc: 0.9832

 ROC-AUC - epoch: 3 - score: 0.985083

Epoch 00003: val_loss improved from 0.04528 to 0.04239, saving model to best_model.hdf5
Epoch 4/5
143613/143613 [=

In [0]:
pred = model.predict(x_test, batch_size = 1024, verbose = 1)

153164/153164 [==============================] - 50s 323us/step


In [0]:
submission = pd.read_csv("./sample_submission.csv")
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = (pred)
submission.to_csv("submission_model3_Feb23_auc_988480.csv", index = False)

In [0]:
##Next tries:
#https://www.kaggle.com/tunguz/bi-gru-cnn-poolings-gpu-kernel-version
#https://www.kaggle.com/c/quora-insincere-questions-classification/discussion/80568 